In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

In [220]:
class RNN:
    # Weights
    W = []
    # Layers
    M = []
    # Bias
    b = []
    # Activation function for each layer
    A = []
    Yhat = []
    costs = []
    lossType = 'mse'
    optimizer = 'none'
    
    def __init__(self):
        self.reset()
    
    def __sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))
    
    def __tanh(self, Z):
        return np.tanh(Z)
    
    def __relu(self,Z):
        return Z * (Z > 0)
    
    # Calls specified activation function
    def __actf(self,Z,ty):
        if ty == 'sigmoid':
            return self.__sigmoid(Z)
        elif ty == 'tanh':
            return self.__tanh(Z)
        elif ty == 'softmax':
            return self.__softmax(Z)
        elif ty == 'none':
            return Z
        else:# ty == 'relu':
            return self.__relu(Z)
        
        # Calls specified activation function derivative
    def __actf_dv(self,Z,ty):
        if ty == 'sigmoid':
            return Z*(1-Z)
        elif ty == 'tanh':
            return (1-Z*Z)
        elif ty == 'none':
            return Z
        else:# ty == 'relu':
            return np.where(Z > 0, 1, 0)
        
    def __softmax(self,A):
        expA = np.exp(A)
        return expA / expA.sum(axis=1,keepdims=True)
    
    # Cross-entropy cost for softmax
    def __cost(self,T,Y,ty):
        if ty == 'ce':
            return self.__cross_entropy(T,Y)
        elif ty == 'mse':
            return self.__mean_squared(T,Y)
    
    def __cross_entropy(self,T,Y):
        tot = (-T * np.log(Y))
        return tot.sum()
    
    def __mean_squared(self,T,Y):
        tot = np.square(T - Y)
        return tot.sum()
    
    def classification_rate(self,T):
        Yhat = NeuralNetwork.Yhat
        Yp = np.argmax(Yhat,axis=1)
        print('Classification rate: ', np.mean(T == Yp))
    
    # Adds hidden layer with L nodes, d dropout
    def add_layer(self,L,a='sigmoid'):
        self.M.append(L)
        self.A.append(a)
        
    def __shuffle(self,X,Y):
        assert len(X) == len(Y)
        p = np.random.permutation(len(X))
        return X[p],Y[p]
        
            
    # Parameters(M:Layers,W:Weights,b:bias,A:activation function,D:Dropout)
    def __forward(self,M,W,b,A):
        Y = []
        yp = []
        X = M[0]
        
        for i in range(len(X)):
            
            ht_prev = M[1][i]
            for t in range(len(X[i])):
                ht = self.__actf((X[i][t].dot(W[0]) + ht_prev.dot(W[1]) + b[0]),A[0])
                yp.append(ht.dot(W[2]) + b[1])
                #yp = softmax(yp)
                ht_prev = ht
                
            M[1][i] = ht
            Y.append(yp[i][-1])
        
        Y = np.array(Y).reshape(len(Y),-1)
        M[2] = Y
        return Y,M
    
    def fit(self,X,Y,epochs=20000,batchSize=0,learnR=10e-6,reg=0,lossType='mse',optimizer='none'):
        W = self.W
        b = self.b
        M = self.M
        A = self.A
        
        # Initialize layers for M
        N = X.shape[0]
        M[0] = np.random.randn(N,M[0])
            
        # Add input and layer to M
        K = Y.shape[1]
        M.insert(0,X)
        M.append(np.random.randn(N,K))
        A.append('none')
        l = learnR
        
        # Regulate batch size
        batchSize = min(batchSize, N)
        batchSize = max(batchSize, 1)
            
        # Set weights
        for i in range(len(M)-1):
            if i == (len(M)-2):

                W.append(np.random.randn(M[i].shape[1],K) / np.sqrt(M[i].shape[1] + K))
                b.append(np.random.randn(K) / np.sqrt(K))
            else:
                # input to hidden
                W.append(np.random.randn(M[i][i].shape[1],M[i+1].shape[1]) / np.sqrt(M[i][i].shape[1] + M[i+1].shape[1]))
                b.append(np.random.randn(M[i+1].shape[1]) / np.sqrt(M[i+1].shape[1]))
                # hidden to hidden
                W.append(np.random.randn(M[i+1].shape[1],M[i+1].shape[1]) / np.sqrt(M[i+1].shape[1] + M[i+1].shape[1]))
                b.append(np.random.randn(M[i+1].shape[1]) / np.sqrt(M[i+1].shape[1]))
        # Set cache (if using rmsprop/adam)
        if optimizer == 'rms':
            wCache = []
            bCache = []
            eps = 10e-8
            decay = 0.99
            for i in range(len(M)-1):
                if i == (len(M)-2):
                    wCache.append(np.ones((M[i].shape[1],K)))
                    bCache.append(np.ones((K)))
                else:
                    wCache.append(np.ones((M[i].shape[1],M[i+1].shape[1])))
                    bCache.append(np.ones((M[i+1].shape[1])))
        elif optimizer == 'adam':
            wM = []
            wV = []
            bM = []
            bV = []
            eps = 10e-8
            decay1 = 0.9
            decay2 = 0.999
            for i in range(len(M)-1):
                if i == (len(M)-2):
                    wM.append(np.zeros((M[i].shape[1],K)))
                    wV.append(np.zeros((M[i].shape[1],K)))
                    bM.append(np.zeros((K)))
                    bV.append(np.zeros((K)))
                else:
                    wM.append(np.zeros((M[i].shape[1],M[i+1].shape[1])))
                    wV.append(np.zeros((M[i].shape[1],M[i+1].shape[1])))
                    bM.append(np.zeros((M[i+1].shape[1])))
                    bV.append(np.zeros((M[i+1].shape[1])))
        
        costs = []
        for e in range(epochs):
            iterations = N // batchSize
            #X,Y,y = self.__shuffle(X,Y,y)
            for i in range(iterations):
                start = i * batchSize
                end = (i+1) * batchSize
                batchX, batchY = X[start:end],Y[start:end]
                del M[0]
                M.insert(0,batchX)
                #print(sparceY)
                Yp,M = self.__forward(M,W,b,A)
                
                Z = M
                #cost = self.__cost(Y,Yp)
                cost = self.__cost(batchY,Yp,lossType)
                costs.append(cost)
                

                # Adjust weights
                #S = (Y - Yp)
                S = (batchY - Yp)
                n = len(M)-2
                Zt = S
                for i in range(len(M)-1):
                    # Weight and bias derivative
                    if n == 0:
                        dw1 = Z[n].T.dot(Zt)
                        dw1 = dw1[0][-1]
                        db1 = Zt.sum()
                    dw = Z[1].T.dot(Zt)
                    db = Zt.sum()
                    #print(i, n)
                    #print(f"M[{n}]: {Z[n].shape}, Zt: {Zt.shape}, M.T.dot(Zt): {(Z[n].T.dot(Zt)).shape}, W[{n}]: {W[n].shape}")
                    if optimizer == 'none':
                        if n == 0:
                            W[n] += l * (dw1 - reg*W[n])
                            b[n] += l * (db1 - reg*b[n])
                        #print(f"W[{n+1}]: {W[n+1].shape}, dw: {dw.shape}")
                        W[n+1] += l * (dw - reg*W[n+1])
                        b[n+1] += l * (db - reg*b[n+1])
                    elif optimizer == 'rms':
                        # rmsprop
                        wCache[n] = (decay * wCache[n]) + (1-decay) * np.square(dw)
                        bCache[n] = (decay * bCache[n]) + (1-decay) * np.square(db)
                        wDenominator = np.sqrt(wCache[n]) + eps
                        bDenominator = np.sqrt(bCache[n]) + eps
                        
                        W[n] += l * ((dw/wDenominator) - reg*W[n])
                        b[n] += l * ((db/bDenominator) - reg*b[n])
                    elif optimizer == 'adam':
                        
                        #print(wM.shape)
                        wM[n] = (decay1 * wM[n]) + (1-decay1) * dw
                        wV[n] = (decay1 * wV[n]) + (1-decay1) * np.square(dw)
                        bM[n] = (decay2 * bM[n]) + (1-decay2) * db
                        bV[n] = (decay2 * bV[n]) + (1-decay2) * np.square(db)
                        wMhat = wM[n]/(1-decay1**(epochs+1))
                        wVhat = wV[n]/(1-decay1**(epochs+1))
                        bMhat = bM[n]/(1-decay2**(epochs+1))
                        bVhat = bV[n]/(1-decay2**(epochs+1))
                        wDenom = np.sqrt(wVhat) + eps
                        bDenom = np.sqrt(bVhat) + eps
                        #print(W[n].shape, wM[n].shape, wDenom.shape)
                        W[n] += l * ((wMhat/wDenom) - reg*W[n])
                        b[n] += l * ((bMhat/bDenom) - reg*b[n])
                    
                    if i == 0:
                        #print(f"actf: {A[n]}")
                        Zt = Zt.dot(W[n+1].T)*self.__actf_dv(Z[n],A[n-1])
                    elif i != (len(M)-2):
                        #Update Zt
                        #print("In else")
                        #print(f"actf: {A[n-1]}")
                        Zt = Zt.dot(W[n].T)*self.__actf_dv(Z[n],A[n-1])

                    n -= 1
            if e % 100 == 0:
                print(e,costs[-1])
        self.W = W
        self.b = b
        self.M = M
        self.Yhat = Yp
        self.costs = costs
    
    def predict(self,X):
        W = self.W
        b = self.b
        M = self.M
        A = self.A
        del M[0]
        M.insert(0,X)
        Yp,Z = self.__forward(M,W,b,A,D)
        NeuralNetwork.Yhat = Yp
        return Yp
    
    def plot_cost(self):
        costs = self.costs
        plt.plot(costs)
        plt.show()
    
    def reset(self):
        self.W = []
        self.b = []
        self.M = []
        self.A = []
        self.lossType = 'mse'
        self.optimizer = 'none'
        self.Yhat = []
        self.costs = []

In [221]:
series = np.sin(0.1*np.arange(200))

In [222]:
T = 10
X = []
Y = []
for t in range(len(series) - T):
    x = series[t:t+T]
    X.append(x)
    y = series[t+T]
    Y.append(y)

X = np.array(X).reshape(-1,T,1)
Y = np.array(Y).reshape(len(Y),-1)
N = len(X)

In [223]:
model = RNN()

In [224]:
model.add_layer(15,'tanh')

In [225]:
# (X, Y, loss type, # iterations, batch size, learning rate, regulization)
# Loss type('ce': cross-entropy, 'sce': sparse cross-entropy)
lossType = 'mse'
epochs = 20000
batchSize = N
learnR = 10e-4
reg = 0
optimizer = 'none'
model.fit(X,Y,epochs,batchSize,learnR,lossType=lossType,optimizer=optimizer)

0 138.1562779608925
100 8.864082447633998e+52
200 2.4097914963744495e+106
300 6.551264713866248e+159
400 1.7810283344314965e+213
500 4.841907733226208e+266


C:\Users\justi\anaconda3\lib\site-packages\numpy\core\_methods.py:47: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
C:\Users\justi\AppData\Local\Temp/ipykernel_98376/1966523942.py:67: RuntimeWarning: overflow encountered in square
  tot = np.square(T - Y)
C:\Users\justi\AppData\Local\Temp/ipykernel_98376/1966523942.py:244: RuntimeWarning: invalid value encountered in multiply
  Zt = Zt.dot(W[n+1].T)*self.__actf_dv(Z[n],A[n-1])


600 nan
700 nan
800 nan
900 nan
1000 nan
1100 nan
1200 nan
1300 nan
1400 nan
1500 nan
1600 nan
1700 nan
1800 nan


KeyboardInterrupt: 

In [197]:
W[0].shape

(1, 15)

## Individual methods / testing

In [60]:
def forward(X,M,W,b):
    Y = []
    yp = []
    ht_prev = M[0]
    for i in range(len(X)):
        for t in range(len(X[i])):
            ht = tanh((X[i][t].dot(W[0]) + ht_prev.dot(W[1]) + b[0]))
            yp.append(ht.dot(W[2]) + b[1])
            #yp = softmax(yp)
            ht_prev = ht
                   
        Y.append(yp[i][-1])
    
    Y = np.array(Y).reshape(len(Y),-1)               
    return Y,ht

In [61]:
def mse(T,Y):
    tot = np.square(T - Y)
    return tot.sum()

In [62]:
def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

In [63]:
def tanh(Z):
    return np.tanh(Z)

In [64]:
def actf_dv(Z):
    return Z*(1-Z)

In [65]:
def softmax(A):
    expA = np.exp(A)
    return expA / expA.sum(axis=1,keepdims=True)

In [88]:
series = np.sin(0.1*np.arange(200))

In [89]:
T = 10
D = 1
H = 15
X = []
Y = []
for t in range(len(series) - T):
    x = series[t:t+T]
    X.append(x)
    y = series[t+T]
    Y.append(y)

X = np.array(X).reshape(-1,T,1)
Y = np.array(Y).reshape(len(Y),-1)
N = len(X)

In [90]:
A = np.random.randn(1,10,1)

In [91]:
a = A[0]

In [92]:
a[0]

array([2.04200149])

In [93]:
# Initialize layers for M
M = []
W = []
b = []
M.append(np.zeros((N,H)))

# Add input and layer to M
K = 1
M.insert(0,X)
M.append(np.random.randn(N,K))

In [94]:
# Set weights
W = []
b = []
for i in range(len(M)-1):
    if i == (len(M)-2):
        
        W.append(np.random.randn(M[i].shape[1],K) / np.sqrt(M[i].shape[1] + K))
        b.append(np.random.randn(K) / np.sqrt(K))
    else:
        # input to hidden
        W.append(np.random.randn(M[i][i].shape[1],M[i+1].shape[1]) / np.sqrt(M[i][i].shape[1] + M[i+1].shape[1]))
        b.append(np.random.randn(M[i+1].shape[1]) / np.sqrt(M[i+1].shape[1]))
        # hidden to hidden
        W.append(np.random.randn(M[i+1].shape[1],M[i+1].shape[1]) / np.sqrt(M[i+1].shape[1] + M[i+1].shape[1]))
        b.append(np.random.randn(M[i+1].shape[1]) / np.sqrt(M[i+1].shape[1]))

In [95]:
x = M[0]

In [19]:
b = np.random.randn(1,10,15)

In [22]:
b

array([[[ 0.44200743, -0.6221953 , -1.27410177,  1.26456843,
         -0.64982435,  0.45493263, -0.56078996,  0.03988918,
          0.0397615 ,  0.61914404,  2.36833784, -0.24362468,
          0.22429747, -2.06008626, -0.06760491],
        [ 1.1015764 , -0.49692002, -0.55322785,  1.19348815,
         -1.12109738,  0.3068998 , -0.8551629 ,  1.5157507 ,
          0.38804492, -1.78897041, -0.81887887, -0.26092485,
          0.07020068,  0.85390389,  1.4716075 ],
        [-0.11536702, -0.11996291,  0.23865434,  0.73599471,
          1.96379917, -0.61052491, -0.48869941, -0.65067759,
          0.87323981, -0.17860095, -0.01826198,  1.8690311 ,
          1.2128043 ,  0.88699153,  0.39825832],
        [-1.20107721, -0.24706758,  0.73324711,  1.33829827,
          1.4928222 , -0.96264587, -1.53667949,  0.65303322,
          1.90391769,  2.59029672, -0.28770825,  0.20816797,
          1.15490389, -0.27788582, -0.48417521],
        [ 1.57830648,  0.7344579 ,  0.84168023, -0.18405687,
          0

In [96]:
Yp,ht = forward(M[0],M[1],W,b)

In [75]:
Yp.shape

(190, 1)

In [76]:
ht.shape

(15,)

In [109]:
M[1].shape

(190, 15)

In [97]:
zt = (Y-Yp)

In [98]:
dw = M[1].T.dot(zt)

In [99]:
zt1 = (zt).dot(W[2].T)*M[1]

In [101]:
dw1 = M[1].T.dot(zt1)

In [102]:
zt2 = (zt1).dot(W[1].T)*M[1]

In [103]:
dw2 = M[0].T.dot(zt2)

In [107]:
dw2.shape

(1, 10, 15)

In [135]:
W[2] + dw

array([[ 0.1060014 ],
       [ 0.39556017],
       [ 0.16938157],
       [ 0.01540116],
       [-0.03805512],
       [-0.23079295],
       [ 0.18028551],
       [-0.54922265],
       [ 0.65635429],
       [ 0.12996523],
       [-0.21637776],
       [-0.20214289],
       [-0.19526826],
       [ 0.27936436],
       [ 0.19545811]])

In [118]:
y1 = Y.reshape(N,-1)

In [136]:
re = x[0][1].dot(W[0]) + M[1][0].dot(W[1]) + b[0]

In [32]:
ht_p = M[1]

In [33]:
re3 = sigmoid((M[0][0][0].dot(W[0]) + ht_p[0].dot(W[1]) + b[0]))

In [34]:
re3.shape

(15,)

In [139]:
re1 = re.dot(W[2]) + b[1]

In [141]:
len(re1)

15

In [148]:
re2 = re1.reshape(len(re1),-1)

In [147]:
re1

array([-0.32032633, -0.30753756, -0.06444399, -0.3179915 , -0.35411167,
       -0.43051232, -0.30951572, -0.51114223, -0.1661792 , -0.78485887,
       -0.27389987, -0.02894201,  0.28752142, -0.49888721,  0.10897052])

In [149]:
sof = softmax(re2)

In [150]:
sof

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [152]:
re2

array([[-0.32032633],
       [-0.30753756],
       [-0.06444399],
       [-0.3179915 ],
       [-0.35411167],
       [-0.43051232],
       [-0.30951572],
       [-0.51114223],
       [-0.1661792 ],
       [-0.78485887],
       [-0.27389987],
       [-0.02894201],
       [ 0.28752142],
       [-0.49888721],
       [ 0.10897052]])